In [1]:
import os, sys
PWD = os.getenv ('PWD')


def init_django(project_name=None) :
    os.chdir (PWD)
    project_name = project_name or os.environ.get (project_name)

    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE" ] = "true"
    import django
    django. setup ()
init_django("OnlineShop")

In [2]:
import pandas as pd
import numpy as np

In [3]:
from ProductFeed.models import *
from ProductFeed.views import show_product_info

In [45]:
all_column_name = []

In [47]:
Sizes.objects.all()

<QuerySet [<Sizes: XS>, <Sizes: S>, <Sizes: M>, <Sizes: L>, <Sizes: XL>, <Sizes: XXL>, <Sizes: 37>, <Sizes: 38>, <Sizes: 39>, <Sizes: 40>, <Sizes: 41>, <Sizes: 42>, <Sizes: 43>, <Sizes: 44>, <Sizes: 45>, <Sizes: Universal>]>

In [4]:
column_names = []

In [5]:
for field_name, field_value in Clothes.objects.get(pk=5).__dict__.items():
    column_names.append(field_name)

In [6]:
column_names.remove("_state")
column_names

['id',
 'title',
 'cost',
 'category',
 'photo',
 'brand',
 'color',
 'by_count',
 'is_premium',
 'gender',
 'count_in_stock']

In [7]:
d = {}
for name in column_names:
    d[name] = []
d

{'id': [],
 'title': [],
 'cost': [],
 'category': [],
 'photo': [],
 'brand': [],
 'color': [],
 'by_count': [],
 'is_premium': [],
 'gender': [],
 'count_in_stock': []}

In [8]:
for elem in Clothes.objects.all():
    for field_name, field_value in elem.__dict__.items():
        if field_name in d:
            d[field_name].append(field_value)

In [9]:
df = pd.DataFrame(d)

In [10]:
df

,id,title,cost,category,photo,brand,color,by_count,is_premium,gender,count_in_stock
0,1,Толстовка,3000,Одежда,photos/2025/02/18/Снимок_экрана_2025-02-18_в_2...,Topman,blue,10,False,Male,10
1,2,Рубашка,2200,Одежда,photos/2025/02/18/Снимок_экрана_2025-02-18_в_2...,Mia Mania,white,1,False,Male,10
2,3,Футболка,1200,Одежда,photos/2025/02/18/Снимок_экрана_2025-02-18_в_2...,Mother Russia,color,0,False,Male,10
3,4,Кроссовки,5500,Обувь,photos/2025/02/19/Снимок_экрана_2025-02-19_в_1...,Puma,black,1,False,Male,10
4,5,Кардиган,2250,Одежда,photos/2025/02/19/Снимок_экрана_2025-02-19_в_1...,Sela,brown,0,False,Male,10
5,6,Джемпер,7990,Одежда,photos/2025/02/19/Снимок_экрана_2025-02-19_в_1...,oodji,gray,0,False,Male,10
6,7,Брюки,4200,Одежда,photos/2025/02/19/Снимок_экрана_2025-02-19_в_1...,Zolla,black,0,False,Male,10
7,8,Брюки,5500,Одежда,photos/2025/02/19/Снимок_экрана_2025-02-19_в_1...,Fila,pink,1,False,Male,10
8,9,Пиджак,5749,Одежда,photos/2025/02/19/Снимок_экрана_2025-02-19_в_2...,oodji,black,0,False,Male,10
9,10,Жилет,1899,Одежда,photos/2025/02/19/Снимок_экрана_2025-02-19_в_2...,Calliope,blue,0,False,Male,10


In [28]:
add_fields = ["size", "season", "style", "subcategory", "country"]
for elem in Clothes.objects.all():
    for field_name, field_value in elem.size.__dict__.items():
        print(field_name, field_value)

_constructor_args ((<Clothes: Толстовка>,), {})
creation_counter 201
model <class 'ProductFeed.models.Sizes'>
name None
_db None
_hints {}
instance Толстовка
query_field_name clothes
prefetch_cache_name size
source_field_name clothes
target_field_name sizes
symmetrical False
through <class 'ProductFeed.models.Clothes_size'>
reverse False
source_field ProductFeed.Clothes_size.clothes
target_field ProductFeed.Clothes_size.sizes
core_filters {'clothes__id': 1}
pk_field_names {'clothes': 'id'}
related_val (1,)
_constructor_args ((<Clothes: Рубашка>,), {})
creation_counter 202
model <class 'ProductFeed.models.Sizes'>
name None
_db None
_hints {}
instance Рубашка
query_field_name clothes
prefetch_cache_name size
source_field_name clothes
target_field_name sizes
symmetrical False
through <class 'ProductFeed.models.Clothes_size'>
reverse False
source_field ProductFeed.Clothes_size.clothes
target_field ProductFeed.Clothes_size.sizes
core_filters {'clothes__id': 2}
pk_field_names {'clothes': 'id

In [44]:
for s in Clothes.objects.get(pk=3).size.all():
    print(s.size)

M
L
XL
